# Recurrent Neural Networks (RNNs) for Radio Frequency (RF) Signal Analysis

## 1. Introduction

Recurrent Neural Networks (RNNs) are a class of neural networks designed to process sequential data. Unlike feedforward networks, RNNs maintain a hidden state that carries information across time steps, making them suitable for time-series tasks.

In RF signal processing, signals are naturally sequential (time-domain waveforms), which makes RNNs particularly useful for tasks such as:

- Modulation classification
- Signal detection
- Spectrum monitoring
- Anomaly detection

---

## 2. How RNNs Work

- **Input:** Sequence of feature vectors \( x_1, x_2, ..., x_T \) over time steps \( T \).  
- **Hidden State:** \( h_t = f(W_{xh} x_t + W_{hh} h_{t-1} + b_h) \)  
  The hidden state captures the memory of previous inputs.  
- **Output:** \( y_t = g(W_{hy} h_t + b_y) \)  
  Depending on the task, the network can produce a prediction at each time step or only at the final step.

### Limitations of Vanilla RNNs

- Suffer from vanishing/exploding gradients.
- Struggle to learn long-term dependencies.
- Typically replaced by **GRUs** or **LSTMs** for longer sequences.

---

## 3. GRU and LSTM

**GRU (Gated Recurrent Unit):**
- Uses reset and update gates to control memory.
- Fewer parameters than LSTM, faster to train.
- Handles moderate long-term dependencies.

**LSTM (Long Short-Term Memory):**
- Uses cell state with input, forget, and output gates.
- Better at learning long-term dependencies.
- Slightly more computationally expensive than GRU.

---

## 4. Applying RNNs to RF Signals

RF signals are sequences of samples (e.g., amplitude, phase, or I/Q components). RNNs can model temporal dependencies in these sequences for various tasks:

### Example Applications

1. **Modulation Classification**
   - Input: Sequence of I/Q samples or amplitude values.
   - Output: Predicted modulation scheme (e.g., BPSK, QPSK, QAM).
   - RNN can learn the temporal patterns characteristic of each modulation.

2. **Signal Detection**
   - Detect presence of a specific signal within a noisy channel.
   - Output: Probability of signal presence per time step or sequence.

3. **Anomaly Detection**
   - Identify unusual RF patterns (e.g., jamming, interference).
   - RNNs learn normal temporal patterns; deviations are flagged.

### Data Representation

| Signal Type | Input Format |
|-------------|-------------|
| BPSK        | 1D sequence of real amplitudes (+1/-1) |
| QPSK        | 2D sequence of I/Q values |
| QAM         | Multi-dimensional sequence of constellation components |

---

## 5. Advantages of RNNs for RF Analysis

- Capture temporal dependencies in signals.
- Can handle sequences of variable length.
- Learn features automatically, reducing need for manual feature engineering.

---

## 6. Practical Considerations

- **Sequence Length:** Must match sampling interval and duration of RF snippet.
- **Batching:** Use DataLoader with padding or fixed-length sequences.
- **Model Choice:** Use LSTM/GRU for longer sequences; vanilla RNN suffices for short sequences.
- **GPU Acceleration:** Recommended for large datasets or long sequences.

---

## 7. References

1. Goodfellow, I., Bengio, Y., & Courville, A. (2016). *Deep Learning*. MIT Press.
2. O’Shea, T., & Hoydis, J. (2017). *An Introduction to Deep Learning for the Physical Layer*. IEEE Transactions on Cognitive Communications and Networking.
3. Hochreiter, S., & Schmidhuber, J. (1997). *Long Short-Term Memory*. Neural Computation, 9(8), 1735–1780.
4. Cho, K. et al. (2014). *Learning Phrase Representations using RNN Encoder–Decoder for Statistical Machine Translation*. EMNLP.



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [ ]:
# --------------------
# Synthetic RF Dataset
# --------------------
# The RFDataset class generates toy radio-frequency signals.
# It supports BPSK (1D real-valued samples) and QPSK (2D I/Q samples).
# Each dataset instance returns (signal, label).
class RFDataset(Dataset):
    """
    Torch Dataset for RF signals.
    Generates toy signals with either BPSK or QPSK modulation.
    Each sample is (signal, label).
    """
    def __init__(self, num_samples=1000, seq_len=128):
        half = num_samples // 2
        bpsk = self.generate_rf_data(half, seq_len, "BPSK")
        qpsk = self.generate_rf_data(half, seq_len, "QPSK")

        X = np.concatenate([bpsk, qpsk], axis=0)
        y = np.array([0] * half + [1] * half)

        # Shuffle dataset to mix classes
        idx = np.random.permutation(num_samples)
        self.X, self.y = X[idx], y[idx]

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return torch.tensor(self.X[idx], dtype=torch.float32), torch.tensor(self.y[idx], dtype=torch.long)

    @staticmethod
    def generate_rf_data(num_samples, seq_len, modulation):
        """
        Generate synthetic RF signals.
        - BPSK: +1/-1 values (real).
        - QPSK: maps bit pairs to complex constellation points (I/Q).
        """
        if modulation == "BPSK":
            symbols = np.random.choice([1, -1], size=(num_samples, seq_len))
            return symbols.astype(np.float32)[..., None]  # shape [N, L, 1]
        elif modulation == "QPSK":
            bits = np.random.choice([0, 1], size=(num_samples, seq_len, 2))
            mapping = {(0, 0): 1+1j, (0, 1): -1+1j, (1, 0): 1-1j, (1, 1): -1-1j}
            symbols = np.array([[mapping[tuple(b)] for b in row] for row in bits])
            return np.stack([symbols.real, symbols.imag], axis=-1).astype(np.float32)  # [N, L, 2]
        else:
            raise ValueError("Unsupported modulation")

In [ ]:
# --------------------
# RNN Model
# --------------------
# RFSignalRNN is a simple recurrent neural network for sequence classification.
# It reads input sequences of RF samples and predicts the modulation scheme.
class RFSignalRNN(nn.Module):
    """
    Simple RNN classifier for RF signals.
    Takes sequence input [batch, seq_len, input_dim] and predicts modulation class.
    """
    def __init__(self, input_dim=1, hidden_dim=64, num_layers=1, num_classes=2):
        super(RFSignalRNN, self).__init__()
        self.rnn = nn.RNN(input_size=input_dim,
                          hidden_size=hidden_dim,
                          num_layers=num_layers,
                          batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        out, _ = self.rnn(x)       # [batch, seq_len, hidden_dim]
        out = out[:, -1, :]        # last timestep output
        return self.fc(out)        # [batch, num_classes]

In [ ]:
# --------------------
# Training Routine
# --------------------
# train_model handles dataset preparation, batching, GPU usage, and training loop.
# It reports training loss and test accuracy for each epoch.
def train_model(device="cuda" if torch.cuda.is_available() else "cpu"):
    """
    Train RNN on synthetic RF dataset with GPU acceleration.
    Uses DataLoader for batching and shuffling.
    """
    # Dataset + loaders
    train_ds = RFDataset(num_samples=4000, seq_len=128)
    test_ds = RFDataset(num_samples=1000, seq_len=128)

    train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_ds, batch_size=64, shuffle=False)

    # Model
    input_dim = train_ds.X.shape[-1]
    model = RFSignalRNN(input_dim=input_dim, hidden_dim=64, num_layers=1, num_classes=2).to(device)

    # Loss + optimiser
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    # Training loop
    for epoch in range(10):
        model.train()
        total_loss = 0
        for X, y in train_loader:
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            outputs = model(X)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * X.size(0)

        # Validation
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for X, y in test_loader:
                X, y = X.to(device), y.to(device)
                preds = model(X).argmax(dim=1)
                correct += (preds == y).sum().item()
                total += y.size(0)

        acc = correct / total
        print(f"Epoch {epoch+1}: Train Loss={total_loss/len(train_ds):.4f}, Test Acc={acc:.3f}")

In [ ]:
# --------------------
# Script Entry Point
# --------------------
# The script starts here: runs the training routine.
if __name__ == "__main__":
    train_model()
